In [1]:
import sqlite3 as sql

import numpy as np
import pandas as pd
from pandas import cut, read_sql
import pickle as pkl
from random import choices
from sklearn import metrics
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy.optimize as optimize
from standard_plotting import set_size

In [2]:

# indir_track_cascade_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/old_morten_peter_db/track_mu_Track_cascade_whole_MP_db.csv"
# indir_energy_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/old_morten_peter_db/energy_Energy_Whole_MP.csv"
# indir_zenith_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/old_morten_peter_db/zenith_Zenith_Whole_MP.csv"
# indir_azimuth_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/old_morten_peter_db/azimuth_Azimuth_Whole_MP.csv"
# indir_multiclass_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/old_morten_peter_db/pid_Multiclass_try_2_on_old_db_trained_on_new_muons.csv"
# indir_position_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/old_morten_peter_db/['position_x', 'position_y', 'position_z']_Position_Whole_MP.csv"
# indir_osc_weight_and_lvl3_bool_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/old_morten_peter_db/Osc_weight_and_lvl3_bool.csv"
indir_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/track_cascade_sets/Old_muon_more_noise_Monte_Carlo_all_events_all_variables.csv"


In [3]:
# track_cascade_MC = pd.read_csv(indir_track_cascade_MC).sort_values('event_no').reset_index(drop = True)
# energy_MC = pd.read_csv(indir_energy_MC).sort_values('event_no').reset_index(drop = True)
# zenith_MC = pd.read_csv(indir_zenith_MC).sort_values('event_no').reset_index(drop = True)
# azimuth_MC = pd.read_csv(indir_azimuth_MC).sort_values('event_no').reset_index(drop = True)
# multiclass_MC = pd.read_csv(indir_multiclass_MC).sort_values('event_no').reset_index(drop = True)
# position_MC = pd.read_csv(indir_position_MC).sort_values('event_no').reset_index(drop = True)
# osc_weight_and_lvl3_bool_MC = pd.read_csv(indir_osc_weight_and_lvl3_bool_MC).sort_values('event_no').reset_index(drop = True)
MC = pd.read_csv(indir_MC).sort_values('event_no').reset_index(drop = True)

In [4]:
# indir_db_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/osc_next_database_Peter_and_Morten/merged_database/osc_next_level3_v2.00_genie_muongun_noise_120000_140000_160000_130000_888003.db"
# with sql.connect(indir_db_MC) as con:
#          query = f"""
#          SELECT
#              event_no, pid, interaction_type
#          FROM 
#              truth
#          """
#          MC_truth = read_sql(query,con)
# MC_truth = MC_truth.sort_values('event_no').reset_index(drop = True)

In [5]:
# track_mu = []
# for i in range(len(MC_truth)):
#     if (MC_truth['pid'][i] == -14 or MC_truth['pid'][i] == 14) and MC_truth['interaction_type'][i] == 1:
#         track_mu.append(1)
#     else:
#         track_mu.append(0)

In [6]:
# MC_truth['track_mu'] = np.array(track_mu)

In [8]:
# print(np.shape(MC))
# total_MC = pd.merge(MC.reset_index(drop=True),MC_truth.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
# print(np.shape(total_MC))

(30796130, 27)
(30796130, 30)


In [9]:
# total_MC_no_neutrinos = total_MC[total_MC['pid'].isin((-1,1,-13,13))]

KeyError: 'pid'

In [11]:
indir_MC_130000_muons_neutrinos_and_noise = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/track_cascade_sets/Old_muon_more_noise_Monte_Carlo_all_events_all_variables.csv"
MC = pd.read_csv(indir_MC_130000_muons_neutrinos_and_noise).sort_values('event_no').reset_index(drop = True)

In [13]:
total_MC = pd.merge(MC.reset_index(drop=True),MC_truth.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)


In [15]:
total_MC.columns

Index(['track_mu_pred', 'event_no', 'energy_pred', 'zenith_pred',
       'zenith_kappa', 'azimuth_pred', 'azimuth_kappa', 'pid_noise_pred',
       'pid_muon_pred', 'pid_neutrino_pred', 'pid_x',
       'pid_neutrino_pred_logit', 'position_x_pred', 'position_y_pred',
       'position_z_pred', 'RunID', 'EventID', 'SubrunID', 'SubEventID',
       'total_osc_weight', 'L3_oscNext_bool', 'azimuth', 'zenith', 'energy',
       'position_z', 'position_y', 'position_x', 'pid_y', 'interaction_type',
       'track_mu'],
      dtype='object')

In [20]:
total_MC.drop(columns=['pid_y','interaction_type'],inplace=True)

In [21]:
total_MC.rename(columns={"pid_x": "pid"},inplace=True)

In [22]:
total_MC_no_neutrinos = total_MC[total_MC['pid'].isin((-1,1,-13,13))]

In [23]:
total_MC_no_neutrinos.columns

Index(['track_mu_pred', 'event_no', 'energy_pred', 'zenith_pred',
       'zenith_kappa', 'azimuth_pred', 'azimuth_kappa', 'pid_noise_pred',
       'pid_muon_pred', 'pid_neutrino_pred', 'pid', 'pid_neutrino_pred_logit',
       'position_x_pred', 'position_y_pred', 'position_z_pred', 'RunID',
       'EventID', 'SubrunID', 'SubEventID', 'total_osc_weight',
       'L3_oscNext_bool', 'azimuth', 'zenith', 'energy', 'position_z',
       'position_y', 'position_x', 'track_mu'],
      dtype='object')

In [24]:
outdir = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/track_cascade_sets/"

total_MC_no_neutrinos.to_csv(outdir + 'Old_muon_more_noise_Monte_Carlo_all_events_all_variables.csv',index=False)

In [25]:
np.shape(total_MC_no_neutrinos[total_MC_no_neutrinos['pid_neutrino_pred_logit']>1])

(469794, 28)

In [26]:
small_MC = total_MC_no_neutrinos[total_MC_no_neutrinos['pid_neutrino_pred_logit']>1]
small_MC.to_csv(outdir + 'Old_muon_more_noise_Monte_Carlo_neutrino_prob_logit_above_1_all_variables.csv',index=False)

In [27]:

small_small_MC = total_MC_no_neutrinos[total_MC_no_neutrinos['pid_neutrino_pred_logit']>12]
small_small_MC.to_csv(outdir + 'Old_muon_more_noise_Monte_Carlo_neutrino_prob_logit_above_12_all_variables.csv',index=False)

In [4]:
# def to_logit(p):
#     eps = 0.0000001
#     try:
#         if np.isnan(p):
#             return
#         p = p*(1-2*eps)+eps
#         logit = np.log(p/(1-p))
#     except ZeroDivisionError as e:
#         print(e)
#     return logit

# multiclass_MC['pid_neutrino_pred_logit'] = pd.Series(multiclass_MC['pid_neutrino_pred']).apply(to_logit)

In [5]:
# osc_weight_and_lvl3_bool_MC['osc_weight'] = osc_weight_and_lvl3_bool_MC['osc_weight'].fillna(0)

In [6]:
# nr_electron_neutrino_i3_files = 602
# nr_muon_neutrino_i3_files = 1518
# nr_tau_neutrino_i3_files = 334
# nr_noise_i3_files = 7001

# nr_muon_i3_files = 11230 - 7002-334-1518-602


# nr_electron_neutrinos_total = 8118575 + 183333
# nr_muon_neutrinos_total = 19923619 + 183333
# nr_tau_neutrinos_total = 8702748 + 183333
# nr_muon_total = 15990173
# nr_noise_total = 15186239

# nr_electron_neutrinos_test =8118575 + 183333
# nr_muon_neutrinos_test = 19923619 + 183333
# nr_tau_neutrinos_test =8702748 + 183333
# nr_muon_test = nr_muon_total
# nr_noise_test = nr_noise_total

# electron_neutrino_add_weight = 1/(nr_electron_neutrino_i3_files*nr_electron_neutrinos_test/nr_electron_neutrinos_total)
# muon_neutrino_add_weight = 1/(nr_muon_neutrino_i3_files*nr_muon_neutrinos_test/nr_muon_neutrinos_total)
# tau_neutrino_add_weight = 1/(nr_tau_neutrino_i3_files*nr_tau_neutrinos_test/nr_tau_neutrinos_total)
# muon_add_weight = 1/(nr_muon_i3_files*nr_muon_test/nr_muon_total)
# noise_add_weight = 1/(nr_noise_i3_files*nr_noise_test/nr_noise_total)
# print(electron_neutrino_add_weight,muon_neutrino_add_weight,tau_neutrino_add_weight)

# electron_neutrino_event_nos = multiclass_MC['event_no'][multiclass_MC['pid'].isin((-12,12))].to_list()
# muon_neutrino_event_nos = multiclass_MC['event_no'][multiclass_MC['pid'].isin((-14,14))].to_list()
# tau_neutrino_event_nos = multiclass_MC['event_no'][multiclass_MC['pid'].isin((-16,16))].to_list()
# muon_event_nos = multiclass_MC['event_no'][multiclass_MC['pid'].isin((-13,13))].to_list()
# noise_event_nos = multiclass_MC['event_no'][multiclass_MC['pid'].isin((-1,1))].to_list()
# neutrino_event_nos = multiclass_MC['event_no'][multiclass_MC['pid'].isin((-12,12,-14,14,-16,16))].to_list()

# print(osc_weight_and_lvl3_bool_MC['osc_weight'][osc_weight_and_lvl3_bool_MC['event_no'].isin((electron_neutrino_event_nos))].head(10))
# osc_weight_and_lvl3_bool_MC['osc_weight'][osc_weight_and_lvl3_bool_MC['event_no'].isin((electron_neutrino_event_nos))] *= electron_neutrino_add_weight
# print(osc_weight_and_lvl3_bool_MC['osc_weight'][osc_weight_and_lvl3_bool_MC['event_no'].isin((electron_neutrino_event_nos))].head(10))
# osc_weight_and_lvl3_bool_MC['osc_weight'][osc_weight_and_lvl3_bool_MC['event_no'].isin((muon_neutrino_event_nos))] *= muon_neutrino_add_weight
# osc_weight_and_lvl3_bool_MC['osc_weight'][osc_weight_and_lvl3_bool_MC['event_no'].isin((tau_neutrino_event_nos))] *= tau_neutrino_add_weight
# osc_weight_and_lvl3_bool_MC['osc_weight'][osc_weight_and_lvl3_bool_MC['event_no'].isin((muon_event_nos))] *= muon_add_weight
# osc_weight_and_lvl3_bool_MC['osc_weight'][osc_weight_and_lvl3_bool_MC['event_no'].isin((noise_event_nos))] *= noise_add_weight

In [7]:
# print(np.shape(track_cascade_MC))
# print(np.shape(energy_MC))
# print(np.shape(zenith_MC))
# print(np.shape(azimuth_MC))
# print(np.shape(multiclass_MC))
# print(np.shape(position_MC))
# print(np.shape(osc_weight_and_lvl3_bool_MC))

In [8]:
# print((track_cascade_MC.columns))
# print((energy_MC.columns))
# print((zenith_MC.columns))
# print((azimuth_MC.columns))
# print((multiclass_MC.columns))
# print((position_MC.columns))
# print((osc_weight_and_lvl3_bool_MC.columns))

In [9]:
# track_cascade_MC.drop(columns=['Unnamed: 0'],inplace=True)
# energy_MC.drop(columns=['Unnamed: 0'],inplace=True)
# zenith_MC.drop(columns=['Unnamed: 0'],inplace=True)
# azimuth_MC.drop(columns=['Unnamed: 0'],inplace=True)
# multiclass_MC.drop(columns=['Unnamed: 0'],inplace=True)
# position_MC.drop(columns=['Unnamed: 0'],inplace=True)
#osc_weight_and_lvl3_bool_MC.drop(columns=['Unnamed: 0'],inplace=True)

In [10]:
# indir_db_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/osc_next_database_Peter_and_Morten/merged_database/osc_next_level3_v2.00_genie_muongun_noise_120000_140000_160000_130000_888003.db"
# with sql.connect(indir_db_MC) as con:
#          query = f"""
#          SELECT
#              event_no, azimuth, zenith, energy, position_x, position_y, position_z
#          FROM 
#              truth
#          """
#          MC_truths = read_sql(query,con)
# MC_truths = MC_truths.sort_values('event_no').reset_index(drop = True)

In [11]:
indir_db_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/osc_next_database_Peter_and_Morten/merged_database/osc_next_level3_v2.00_genie_muongun_noise_120000_140000_160000_130000_888003.db"
with sql.connect(indir_db_MC) as con:
         query = f"""
         SELECT
             event_no, position_x
         FROM 
             truth
         """
         MC_pos_x = read_sql(query,con)
MC_pos_x = MC_pos_x.sort_values('event_no').reset_index(drop = True)

In [12]:
# total_MC = pd.merge(track_cascade_MC.reset_index(drop=True),energy_MC.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
# total_MC = pd.merge(total_MC.reset_index(drop=True),zenith_MC.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
# total_MC = pd.merge(total_MC.reset_index(drop=True),azimuth_MC.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
# total_MC = pd.merge(total_MC.reset_index(drop=True),multiclass_MC.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
# total_MC = pd.merge(total_MC.reset_index(drop=True),position_MC.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
# total_MC = pd.merge(total_MC.reset_index(drop=True),osc_weight_and_lvl3_bool_MC.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
# total_MC = pd.merge(total_MC.reset_index(drop=True),MC_truths.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
total_MC = pd.merge(MC.reset_index(drop=True),MC_pos_x.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)


In [16]:
total_MC.columns

Index(['track_mu_pred', 'event_no', 'energy_pred', 'zenith_pred',
       'zenith_kappa', 'azimuth_pred', 'azimuth_kappa', 'pid_noise_pred',
       'pid_muon_pred', 'pid_neutrino_pred', 'pid', 'pid_neutrino_pred_logit',
       'position_x_pred', 'position_y_pred', 'position_z_pred', 'RunID',
       'EventID', 'SubrunID', 'SubEventID', 'total_osc_weight',
       'L3_oscNext_bool', 'azimuth', 'zenith', 'energy', 'position_z',
       'position_y', 'position_z.1', 'position_x'],
      dtype='object')

In [17]:
total_MC.drop(columns=['position_z.1'],inplace=True)

In [18]:
total_MC_no_neutrinos = total_MC

In [13]:
# total_MC_no_neutrinos = total_MC[total_MC['pid'].isin((-1,1,-13,13))]


In [14]:
# total_MC_no_neutrinos.rename(columns={"osc_weight": "total_osc_weight"},inplace=True)

In [ ]:
# print(total_MC_no_neutrinos.columns)

Index(['track_mu_pred', 'event_no', 'energy_pred', 'zenith_pred',
       'zenith_kappa', 'azimuth_pred', 'azimuth_kappa', 'pid_noise_pred',
       'pid_muon_pred', 'pid_neutrino_pred', 'pid', 'pid_neutrino_pred_logit',
       'position_x_pred', 'position_y_pred', 'position_z_pred', 'RunID',
       'EventID', 'SubrunID', 'SubEventID', 'total_osc_weight',
       'L3_oscNext_bool', 'azimuth', 'zenith', 'energy', 'position_z',
       'position_y', 'position_z'],
      dtype='object')


In [19]:
outdir = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/track_cascade_sets/"

total_MC_no_neutrinos.to_csv(outdir + 'Old_muon_more_noise_Monte_Carlo_all_events_all_variables.csv',index=False)

In [ ]:
np.shape(total_MC_no_neutrinos[total_MC_no_neutrinos['pid_neutrino_pred_logit']>1])

(472532, 27)

In [21]:
small_MC = total_MC_no_neutrinos[total_MC_no_neutrinos['pid_neutrino_pred_logit']>1]
small_MC.to_csv(outdir + 'Old_muon_more_noise_Monte_Carlo_neutrino_prob_logit_above_1_all_variables.csv',index=False)

In [22]:

small_small_MC = total_MC_no_neutrinos[total_MC_no_neutrinos['pid_neutrino_pred_logit']>12]
small_small_MC.to_csv(outdir + 'Old_muon_more_noise_Monte_Carlo_neutrino_prob_logit_above_12_all_variables.csv',index=False)

In [23]:
print(np.shape(small_small_MC))

(459, 27)


In [24]:
train_indir = "/groups/icecube/petersen/GraphNetDatabaseRepository/osc_next_database_new_muons_peter/train_val_test_split/Multiclassification/Multiclassification_train_event_no.csv"
val_indir = "/groups/icecube/petersen/GraphNetDatabaseRepository/osc_next_database_new_muons_peter/train_val_test_split/Multiclassification/Multiclassification_val_event_no.csv"

In [25]:
train = pd.read_csv(train_indir).sort_values('event_no').reset_index(drop = True)
val = pd.read_csv(val_indir).sort_values('event_no').reset_index(drop = True)
New_muon_indir = "/groups/icecube/petersen/GraphNetDatabaseRepository/osc_next_database_new_muons_peter/Merged_db/osc_next_level3_v2.00_genie_muongun_noise_120000_140000_160000_139008_888003_retro.db"

indir_db_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/osc_next_database_Peter_and_Morten/merged_database/osc_next_level3_v2.00_genie_muongun_noise_120000_140000_160000_130000_888003.db"
with sql.connect(indir_db_MC) as con:
         query = f"""
         SELECT
             event_no, RunID, EventID, SubrunID, SubEventID
         FROM 
             truth
         """
         new_muon = read_sql(query,con)

new_muon = new_muon.sort_values('event_no').reset_index(drop = True)

In [37]:
new_muon.rename(columns={"event_no": "new_muon_event_no"},inplace=True)


In [38]:
new_muon_train_and_val = new_muon[(new_muon['new_muon_event_no'].isin(train['event_no'].to_list()))|(new_muon['new_muon_event_no'].isin(val['event_no'].to_list()))]


merged = pd.concat([new_muon,])

In [55]:
merged_df = total_MC_no_neutrinos.merge(new_muon_train_and_val[['RunID', 'EventID', 'SubrunID', 'SubEventID']], 
                                        on=['RunID', 'EventID', 'SubrunID', 'SubEventID'], 
                                        how='left', indicator=True)

In [66]:
sum(merged_df['_merge'] =='both')

770340

In [69]:
filtered_df = merged_df.loc[~((merged_df['_merge'] == 'both') & merged_df['pid'].isin((-1,1)))].drop(columns='_merge')

In [70]:
print(filtered_df.columns)
print(len(merged_df))
print(len(new_muon_train_and_val))
print(len(filtered_df))
print(len(total_MC_no_neutrinos)-len(filtered_df))

Index(['track_mu_pred', 'event_no', 'energy_pred', 'zenith_pred',
       'zenith_kappa', 'azimuth_pred', 'azimuth_kappa', 'pid_noise_pred',
       'pid_muon_pred', 'pid_neutrino_pred', 'pid', 'pid_neutrino_pred_logit',
       'position_x_pred', 'position_y_pred', 'position_z_pred', 'RunID',
       'EventID', 'SubrunID', 'SubEventID', 'total_osc_weight',
       'L3_oscNext_bool', 'azimuth', 'zenith', 'energy', 'position_z',
       'position_y', 'position_x'],
      dtype='object')
31176412
1649999
30796130
380282


In [76]:
print(np.sum(filtered_df['pid']==-1))
print(np.sum(total_MC_no_neutrinos['pid']==-1))
print(np.sum(filtered_df['pid']==13))
print(np.sum(total_MC_no_neutrinos['pid']==13))

14805957
15186239
15990173
15990173


In [82]:
print(np.sum(filtered_df['total_osc_weight'][filtered_df['pid'].isin((-1,1))]))
print(np.sum(total_MC_no_neutrinos['total_osc_weight'][total_MC_no_neutrinos['pid'].isin((-1,1))]))

6.624471551454647
6.624471551454506


In [81]:
# filtered_df['total_osc_weight'][filtered_df['pid'].isin((-1,1))] = filtered_df['total_osc_weight'][filtered_df['pid'].isin((-1,1))] *6.624471551454506/6.458586680912811

In [83]:
outdir = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/track_cascade_sets/"

filtered_df.to_csv(outdir + 'Old_muon_more_noise_Monte_Carlo_all_events_all_variables.csv',index=False)

In [84]:
small_MC = filtered_df[filtered_df['pid_neutrino_pred_logit']>1]
small_MC.to_csv(outdir + 'Old_muon_more_noise_Monte_Carlo_neutrino_prob_logit_above_1_all_variables.csv',index=False)
small_small_MC = filtered_df[filtered_df['pid_neutrino_pred_logit']>12]
small_small_MC.to_csv(outdir + 'Old_muon_more_noise_Monte_Carlo_neutrino_prob_logit_above_12_all_variables.csv',index=False)